In [1]:
import mediapipe as mp

2024-11-24 21:05:54.712370: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-24 21:05:54.717411: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-24 21:05:54.734986: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732478754.755636  249034 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732478754.761062  249034 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 21:05:54.783001: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
mphands = mp.solutions.hands

In [3]:
hands = mphands.Hands(static_image_mode=False,
                          max_num_hands=2,
                          model_complexity=1,
                          min_detection_confidence=0,
                          min_tracking_confidence=0.5)

I0000 00:00:1732478845.549269  249034 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1732478845.599569  249295 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 560.35.03), renderer: NVIDIA GeForce GTX 1060/PCIe/SSE2


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1732478845.663631  249284 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732478845.686537  249285 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [4]:
def perform_landmark_tracking_from_images(image_paths, before=True, total_frames=None):
    """
    Process a sequence of images to perform landmark tracking using MediaPipe.

    Args:
        image_paths (list): List of image file paths in sequential order.
        before (bool): Flag to determine the return format.
        total_frames (int, optional): Total number of frames to process.

    Returns:
        Tuple of landmark coordinates arrays.
    """
    # Initialize MediaPipe Hands module
    hands = mphands.Hands(static_image_mode=False,
                          max_num_hands=2,
                          model_complexity=1,
                          min_detection_confidence=0,
                          min_tracking_confidence=0.5)

    # Create an empty list to store landmark coordinates
    num_landmarks = 21  # Since there are 21 landmarks per hand

    total_frames = total_frames or len(image_paths)

    left_x_coords = np.full((total_frames, num_landmarks), np.nan)
    left_y_coords = np.full((total_frames, num_landmarks), np.nan)
    left_z_coords = np.full((total_frames, num_landmarks), np.nan)

    right_x_coords = np.full((total_frames, num_landmarks), np.nan)
    right_y_coords = np.full((total_frames, num_landmarks), np.nan)
    right_z_coords = np.full((total_frames, num_landmarks), np.nan)

    frame_index = 0

    for image_path in image_paths:
        # Read the image file
        image = cv2.imread(image_path)
        if image is None:
            print(f"Warning: Unable to read image {image_path}. Skipping.")
            frame_index += 1
            continue

        # Rotate the image 90 degrees clockwise
        image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)

        # Process the image using the MediaPipe Hands module
        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        if results.multi_hand_landmarks:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                # Identify left or right hand by the handedness
                if handedness.classification[0].label == 'Right':
                    x_coords = left_x_coords
                    y_coords = left_y_coords
                    z_coords = left_z_coords
                elif handedness.classification[0].label == 'Left':
                    x_coords = right_x_coords
                    y_coords = right_y_coords
                    z_coords = right_z_coords
                for i, landmark in enumerate(hand_landmarks.landmark):
                    # Accessing landmark coordinates
                    x = landmark.x
                    y = landmark.y
                    z = landmark.z
                    # Update landmark coordinates in the arrays
                    x_coords[frame_index, i] = x
                    y_coords[frame_index, i] = y
                    z_coords[frame_index, i] = z

        frame_index += 1

        # Exit the loop if the last frame is reached
        if total_frames is not None and frame_index >= total_frames:
            break

    # Transpose the landmark coordinates arrays
    left_x_coords = left_x_coords.T
    left_y_coords = left_y_coords.T
    left_z_coords = left_z_coords.T

    right_x_coords = right_x_coords.T
    right_y_coords = right_y_coords.T
    right_z_coords = right_z_coords.T

    if before:
        return left_x_coords, left_y_coords, left_z_coords, right_x_coords, right_y_coords, right_z_coords
    else:
        return left_x_coords, left_y_coords, left_z_coords, right_x_coords, right_y_coords, right_z_coords

In [5]:
import re

def sorted_natural(file_list):
    """
    Sort files in natural order (e.g., file1, file2, ..., file10).

    Args:
        file_list (list): List of file names.

    Returns:
        list: Naturally sorted list of file names.
    """
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(file_list, key=alphanum_key)

In [9]:
import os
import re
import cv2
import numpy as np
import mediapipe as mp
from pathlib import Path
from tqdm import tqdm  # For progress visualization

In [10]:
def sorted_natural(file_list):
    """
    Sort files in natural order (e.g., file1, file2, ..., file10).

    Args:
        file_list (list): List of file names.

    Returns:
        list: Naturally sorted list of file names.
    """
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(file_list, key=alphanum_key)

In [11]:
def perform_landmark_tracking_from_images(image_paths, before=True, total_frames=None):
    """
    Process a sequence of images to perform landmark tracking using MediaPipe.

    Args:
        image_paths (list): List of image file paths in sequential order.
        before (bool): Flag to determine the return format.
        total_frames (int, optional): Total number of frames to process.

    Returns:
        Tuple containing:
            - List of hand landmark objects per frame.
            - Numpy arrays of landmark coordinates.
    """
    # Initialize MediaPipe Hands module
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False,
                           max_num_hands=2,
                           model_complexity=1,
                           min_detection_confidence=0.5,
                           min_tracking_confidence=0.5)

    # Lists to store landmark objects
    all_hand_landmarks = []

    # Create empty arrays to store landmark coordinates
    num_landmarks = 21  # 21 landmarks per hand
    total_frames = total_frames or len(image_paths)

    left_x_coords = np.full((total_frames, num_landmarks), np.nan)
    left_y_coords = np.full((total_frames, num_landmarks), np.nan)
    left_z_coords = np.full((total_frames, num_landmarks), np.nan)

    right_x_coords = np.full((total_frames, num_landmarks), np.nan)
    right_y_coords = np.full((total_frames, num_landmarks), np.nan)
    right_z_coords = np.full((total_frames, num_landmarks), np.nan)

    frame_index = 0

    for img_path in image_paths:
        image = cv2.imread(img_path)
        if image is None:
            print(f"Warning: Could not read {img_path}. Skipping.")
            all_hand_landmarks.append(None)
            frame_index += 1
            continue

        # Convert the BGR image to RGB before processing.
        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        frame_landmarks = []

        if results.multi_hand_landmarks and results.multi_handedness:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                frame_landmarks.append(hand_landmarks)

                # Determine if the hand is left or right
                hand_label = handedness.classification[0].label.lower()
                if hand_label == 'left':
                    target_x = left_x_coords
                    target_y = left_y_coords
                    target_z = left_z_coords
                else:
                    target_x = right_x_coords
                    target_y = right_y_coords
                    target_z = right_z_coords

                for i, landmark in enumerate(hand_landmarks.landmark):
                    target_x[frame_index, i] = landmark.x
                    target_y[frame_index, i] = landmark.y
                    target_z[frame_index, i] = landmark.z

        all_hand_landmarks.append(frame_landmarks)
        frame_index += 1

        # Exit the loop if the last frame is reached
        if total_frames is not None and frame_index >= total_frames:
            break

    # Release the MediaPipe Hands object
    hands.close()

    if before:
        return (left_x_coords, left_y_coords, left_z_coords,
                right_x_coords, right_y_coords, right_z_coords)
    else:
        return all_hand_landmarks, (left_x_coords, left_y_coords, left_z_coords,
                                    right_x_coords, right_y_coords, right_z_coords)

In [12]:
def overlay_landmarks(image, hand_landmarks, connections=mp.solutions.hands.HAND_CONNECTIONS):
    """
    Draw MediaPipe landmarks on the image.

    Args:
        image (numpy.ndarray): The original image.
        hand_landmarks (list): List of hand landmark objects.
        connections (list): List of connections between landmarks.

    Returns:
        numpy.ndarray: Annotated image with landmarks.
    """
    if hand_landmarks:
        mp_drawing = mp.solutions.drawing_utils
        for landmarks in hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                landmarks,
                connections,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )
    return image

In [13]:
def images_to_video(image_paths, output_video_path, fps=1):
    """
    Compile a sequence of images into a video file.

    Args:
        image_paths (list): List of image file paths in order.
        output_video_path (str): Path to save the output video.
        fps (int, optional): Frames per second for the output video. Defaults to 1.

    Returns:
        None
    """
    if not image_paths:
        print("No images to compile into video.")
        return

    # Read the first image to get the size
    first_image = cv2.imread(image_paths[0])
    if first_image is None:
        print(f"Error reading {image_paths[0]}.")
        return

    height, width, layers = first_image.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can use other codecs as needed
    video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    for img_path in image_paths:
        img = cv2.imread(img_path)
        if img is not None:
            video.write(img)
        else:
            print(f"Warning: Skipping {img_path} as it could not be read.")

    video.release()
    print(f"Video saved to {output_video_path}")

In [16]:
def process_thermal_dataset(dataset_path, visualization_output_path, fps=1):
    """
    Traverse through the dataset, perform landmark tracking, visualize results, and save videos.

    Args:
        dataset_path (str): Path to the thermal dataset directory.
        visualization_output_path (str): Path to save visualization videos.
        fps (int, optional): Frames per second for the output videos. Defaults to 1.

    Returns:
        None
    """
    # Ensure the visualization output directory exists
    os.makedirs(visualization_output_path, exist_ok=True)

    # Iterate through each class directory
    for class_dir in tqdm(os.listdir(dataset_path), desc="Processing Classes"):
        class_path = os.path.join(dataset_path, class_dir)
        if not os.path.isdir(class_path):
            continue

        # Iterate through each take directory within the class
        for take_dir in os.listdir(class_path):
            take_path = os.path.join(class_path, take_dir)
            if not os.path.isdir(take_path):
                continue

            # List all image files in the take directory
            image_files = [f for f in os.listdir(take_path) if f.lower().endswith('.jpg')]
            if not image_files:
                print(f"No images found in {take_path}. Skipping.")
                continue

            # Sort image files in natural order
            sorted_images = sorted_natural(image_files)
            image_paths = [os.path.join(take_path, img) for img in sorted_images]

            # Perform landmark tracking and get landmark objects
            hand_landmarks_list, landmarks = perform_landmark_tracking_from_images(image_paths, before=False)

            # Visualize landmarks on images
            annotated_image_paths = []
            for idx, img_path in enumerate(image_paths):
                img = cv2.imread(img_path)
                if img is None:
                    print(f"Warning: Could not read {img_path}. Skipping visualization for this frame.")
                    continue

                # Retrieve landmarks for the current frame
                frame_hand_landmarks = hand_landmarks_list[idx]

                # Overlay landmarks
                annotated_img = overlay_landmarks(img.copy(), frame_hand_landmarks)
                annotated_img_path = os.path.join(take_path, f"annotated_{os.path.basename(img_path)}")
                cv2.imwrite(annotated_img_path, annotated_img)
                annotated_image_paths.append(annotated_img_path)

            # Compile annotated images into a video
            if annotated_image_paths:
                # Create corresponding visualization directory structure
                class_vis_path = os.path.join(visualization_output_path, class_dir)
                os.makedirs(class_vis_path, exist_ok=True)

                video_filename = f"{take_dir}.mp4"
                output_video_path = os.path.join(class_vis_path, video_filename)
                images_to_video(annotated_image_paths, output_video_path, fps=fps)

                # Optionally, remove annotated images after creating the video
                for annotated_img in annotated_image_paths:
                    os.remove(annotated_img)
                print(f"Visualization video saved for {take_path}")
            else:
                print(f"No annotated images to compile for {take_path}")

In [17]:
if __name__ == "__main__":
    dataset_directory = "/home/user/Desktop/RP/experiments/video_dataset15/thermal"
    visualization_directory = "/home/user/Desktop/RP/experiments/video_dataset15/visualisations"
    process_thermal_dataset(dataset_directory, visualization_directory, fps=1)

Processing Classes:   0%|          | 0/9 [00:00<?, ?it/s]I0000 00:00:1732483652.086619  249034 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1732483652.102211  259788 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 560.35.03), renderer: NVIDIA GeForce GTX 1060/PCIe/SSE2
W0000 00:00:1732483652.137753  259779 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732483652.157164  259784 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video saved to /home/user/Desktop/RP/experiments/video_dataset15/visualisations/FramesClass8/Class8-Take10-007-170223.mp4
Visualization video saved for /home/user/Desktop/RP/experiments/video_dataset15/thermal/FramesClass8/Class8-Take10-007-170223


I0000 00:00:1732483652.661180  249034 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1732483652.674449  259808 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 560.35.03), renderer: NVIDIA GeForce GTX 1060/PCIe/SSE2
W0000 00:00:1732483652.712089  259799 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732483652.733022  259803 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video saved to /home/user/Desktop/RP/experiments/video_dataset15/visualisations/FramesClass8/Class8-Take8-005-130223.mp4
Visualization video saved for /home/user/Desktop/RP/experiments/video_dataset15/thermal/FramesClass8/Class8-Take8-005-130223


I0000 00:00:1732483653.309466  249034 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1732483653.321629  259839 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 560.35.03), renderer: NVIDIA GeForce GTX 1060/PCIe/SSE2
W0000 00:00:1732483653.348226  259827 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732483653.375900  259831 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


...

Video saved to /home/user/Desktop/RP/experiments/video_dataset15/visualisations/FramesClass6/Class6-Take7-005-130223.mp4
Visualization video saved for /home/user/Desktop/RP/experiments/video_dataset15/thermal/FramesClass6/Class6-Take7-005-130223


I0000 00:00:1732484226.825538  249034 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1732484226.837669  292231 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 560.35.03), renderer: NVIDIA GeForce GTX 1060/PCIe/SSE2
W0000 00:00:1732484226.845730  292219 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732484226.854037  292221 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video saved to /home/user/Desktop/RP/experiments/video_dataset15/visualisations/FramesClass6/Class6-Take7-007-170223.mp4
Visualization video saved for /home/user/Desktop/RP/experiments/video_dataset15/thermal/FramesClass6/Class6-Take7-007-170223


I0000 00:00:1732484227.427572  249034 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1732484227.439954  292261 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 560.35.03), renderer: NVIDIA GeForce GTX 1060/PCIe/SSE2
W0000 00:00:1732484227.450659  292249 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732484227.459287  292251 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Processing Classes: 100%|██████████| 9/9 [09:35<00:00, 64.00s/it]

Video saved to /home/user/Desktop/RP/experiments/video_dataset15/visualisations/FramesClass6/Class6-Take2-008-170223.mp4
Visualization video saved for /home/user/Desktop/RP/experiments/video_dataset15/thermal/FramesClass6/Class6-Take2-008-170223
